In [58]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

In [28]:
# vamos a traer el data a un df
data = pd.read_csv('./data/customer_segmentation.csv')

In [29]:
# un primer vistazo a ver que hay
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


directamente de kaggle, una definicion de las features para orientarme

Id: Unique identifier for each individual in the dataset.  
Year_Birth: The birth year of the individual.  
Education: The highest level of education attained by the individual.  
Marital_Status: The marital status of the individual.  
Income: The annual income of the individual.  
Kidhome: The number of young children in the household.  
Teenhome: The number of teenagers in the household.  
Dt_Customer: The date when the customer was first enrolled or became a part of the company's database.  
Recency: The number of days since the last purchase or interaction.  
MntWines: The amount spent on wines.  
MntFruits: The amount spent on fruits.  
MntMeatProducts: The amount spent on meat products.  
MntFishProducts: The amount spent on fish products.  
MntSweetProducts: The amount spent on sweet products.  
MntGoldProds: The amount spent on gold products.  
NumDealsPurchases: The number of purchases made with a discount or as part of a deal.  
NumWebPurchases: The number of purchases made through the company's website.  
NumCatalogPurchases: The number of purchases made through catalogs.  
NumStorePurchases: The number of purchases made in physical stores.  
NumWebVisitsMonth: The number of visits to the company's website in a month.  
AcceptedCmp3: Binary indicator (1 or 0) whether the individual accepted the third marketing campaign.  
AcceptedCmp4: Binary indicator (1 or 0) whether the individual accepted the fourth marketing campaign.  
AcceptedCmp5: Binary indicator (1 or 0) whether the individual accepted the fifth marketing campaign.  
AcceptedCmp1: Binary indicator (1 or 0) whether the individual accepted the first marketing campaign.  
AcceptedCmp2: Binary indicator (1 or 0) whether the individual accepted the second marketing campaign.  
Complain: Binary indicator (1 or 0) whether the individual has made a complaint.  
Z_CostContact: A constant cost associated with contacting a customer.  
Z_Revenue: A constant revenue associated with a successful campaign response.  
Response: Binary indicator (1 or 0) whether the individual responded to the marketing campaign.

In [30]:
# voy a poner esta ultima info en un diccionario para tener a mano esta definiciones

dict_features = {
'Id': 'Unique identifier for each individual in the dataset',
'Year_Birth': 'The birth year of the individual',
'Education': 'The highest level of education attained by the individual',
'Marital_Status': 'The marital status of the individual',
'Income': 'The annual income of the individual',
'Kidhome': 'The number of young children in the household',
'Teenhome': 'The number of teenagers in the household',
'Dt_Customer': 'The date when the customer was first enrolled or became a part of the companys database',
'Recency': 'The number of days since the last purchase or interaction',
'MntWines': 'The amount spent on wines',
'MntFruits': 'The amount spent on fruits',
'MntMeatProducts': 'The amount spent on meat products',
'MntFishProducts': 'The amount spent on fish products',
'MntSweetProducts': 'The amount spent on sweet products',
'MntGoldProds': 'The amount spent on gold products',
'NumDealsPurchases': 'The number of purchases made with a discount or as part of a deal',
'NumWebPurchases': 'The number of purchases made through the companys website',
'NumCatalogPurchases': 'The number of purchases made through catalogs',
'NumStorePurchases': 'The number of purchases made in physical stores',
'NumWebVisitsMonth': 'The number of visits to the companys website in a month',
'AcceptedCmp3': 'Binary indicator (1 or 0) whether the individual accepted the third marketing campaign',
'AcceptedCmp4': 'Binary indicator (1 or 0) whether the individual accepted the fourth marketing campaign',
'AcceptedCmp5': 'Binary indicator (1 or 0) whether the individual accepted the fifth marketing campaign',
'AcceptedCmp1': 'Binary indicator (1 or 0) whether the individual accepted the first marketing campaign',
'AcceptedCmp2': 'Binary indicator (1 or 0) whether the individual accepted the second marketing campaign',
'Complain': 'Binary indicator (1 or 0) whether the individual has made a complaint',
'Z_CostContact': 'A constant cost associated with contacting a customer',
'Z_Revenue': 'A constant revenue associated with a successful campaign response',
'Response': 'Binary indicator (1 or 0) whether the individual responded to the marketing campaign'
}

In [31]:
data.info()

# el data parece bastante limpio, la columna income tiene algunos nulos, pocos, sustiturire seguramente por la media o la mediana, a ver el describe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [32]:
# en referencia a la columna income, la media y la mediana estan bastante parejas, en un primer momento iba a utilizar la mediana por habito personal pero viendo la similitud entre ellas
# finalmente voy a usar la media para rellenar nulos

# minimos y maximos muy dispares, esto va a haber que estandarizarlo como presuponia

# tengo columnas binarias, no se si pasarlas a booleanas o dejarlas en numericas, mas adelante la idea es estandarizar, ademas , para clusterizar necesitare que todas las dimensiones se manejen en los mismos valores (numericos)?
# supongo que si (he consultado y si, debo meverme en valores nomericos).

# tengo 3 features categoricas que voy a tener que pasar a numericas

categoricas = ['Education', 'Marital_Status', 'Dt_Customer']

data.describe()

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.000000,2240.000000,2216.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.0,2240.0,2240.000000
mean,5592.159821,1968.805804,52247.251354,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,37.525446,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
std,3246.662198,11.984069,25173.076661,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,54.628979,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274
min,0.000000,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,2828.250000,1959.000000,35303.000000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,5458.500000,1970.000000,51381.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,8427.750000,1977.000000,68522.000000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
max,11191.000000,1996.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.0,11.0,1.000000


In [33]:
# resuelvo nulos

data.Income.fillna(data.Income.mean(), inplace= True)

C:\Users\franc\AppData\Local\Temp\ipykernel_10500\1086513268.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.Income.fillna(data.Income.mean(), inplace= True)


In [34]:
for feature in categoricas:
    print(feature)
    print(data[feature].value_counts())
    print('-'*10)

Education
Education
Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: count, dtype: int64
----------
Marital_Status
Marital_Status
Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64
----------
Dt_Customer
Dt_Customer
31-08-2012    12
12-09-2012    11
14-02-2013    11
12-05-2014    11
20-08-2013    10
              ..
05-08-2012     1
18-11-2012     1
25-05-2013     1
14-04-2013     1
09-01-2014     1
Name: count, Length: 663, dtype: int64
----------


In [35]:
# Dt_Customer parece que son fechas
# esta columna vamos a tener que trabajarla

dict_features['Dt_Customer']

'The date when the customer was first enrolled or became a part of the companys database'

In [36]:
dates = [categoricas.pop(-1)] # tengo la feature de fechas en la lista de categoricas y no me gusta, voy a ponerla en su propia lista de features fechas 

In [37]:
'''
para las categoricas me encuentro en una disyuntiva que resolvere tal que asi pues considero que es lo correcto:

.- Education: parece una feature donde los valores son ordinales, presentan una importancia acorde a su valor, no es igual tener el graduado que un doctorado, para esta
feature utilizare un objeto ordinal encoder 

.- Marital Status: parece una feature donde los valores son nominales, no presentan una importancia acorde a su valor, no podemos ponderar mejor o peor estar casado que soltero que viudo etc. 
para esta feature utilizare un objeto onehotencoder
'''


'\npara las categoricas me encuentro en una disyuntiva que resolvere tal que asi pues considero que es lo correcto:\n\n.- Education: parece una feature donde los valores son ordinales, presentan una importancia acorde a su valor, no es igual tener el graduado que un doctorado, para esta\nfeature utilizare un objeto ordinal encoder \n\n.- Marital Status: parece una feature donde los valores son nominales, no presentan una importancia acorde a su valor, no podemos ponderar mejor o peor estar casado que soltero que viudo etc. \npara esta feature utilizare un objeto onehotencoder\n'

In [38]:
data['Education'].unique()

# vamos a ver que orden sigue esto

array(['Graduation', 'PhD', 'Master', 'Basic', '2n Cycle'], dtype=object)

In [39]:
dict_features['Education']

# la definicion no ayuda mucho, vamos a mirar por la red, porque a mi PhD (por ejemplo) me dice poco y no se ponderar las categorias

'The highest level of education attained by the individual'

In [40]:
sorted_education = [['Basic', '2n Cycle', 'Graduation', 'Master', 'PhD']] # lista anidada

# ahi lo tenemos ordenado, que cosas, al final PhD es un doctorado que precisamente puse como ejemplo un par de celdas atras 

In [41]:
oh_encoder = OneHotEncoder()
o_encoder = OrdinalEncoder(categories= sorted_education)

# instanciamos los transformadores

In [42]:
# vamos a transformar la feature Education

data['Education'] = o_encoder.fit_transform(data[['Education']])

In [43]:
data['Education'].value_counts() # se ha transformado ok, he cotejado con el value de la feature anterior

Education
2.0    1127
4.0     486
3.0     370
1.0     203
0.0      54
Name: count, dtype: int64

In [44]:
# vamos a transformar la feature Marital_Status

# oh_encoder.fit_transform(data[['Marital_Status']])

# para utilizar el objeto one hot de sk learn tengo que hacer el array, pasarlo a df y concatenarlo al df anterior, no es dificil pero es trabajo innecesario, voy a utilizar get_dumies de pandas que es automatico
# dejo estas lineas de codigo solo para probar que pase por aqui

<2240x8 sparse matrix of type '<class 'numpy.float64'>'
	with 2240 stored elements in Compressed Sparse Row format>

In [48]:
data = pd.get_dummies(data, columns= ['Marital_Status'])

In [49]:
data

,ID,Year_Birth,Education,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,Z_Revenue,Response,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
0,5524,1957,2.0,58138.0,0,0,04-09-2012,58,635,88,...,11,1,False,False,False,False,True,False,False,False
1,2174,1954,2.0,46344.0,1,1,08-03-2014,38,11,1,...,11,0,False,False,False,False,True,False,False,False
2,4141,1965,2.0,71613.0,0,0,21-08-2013,26,426,49,...,11,0,False,False,False,False,False,True,False,False
3,6182,1984,2.0,26646.0,1,0,10-02-2014,26,11,4,...,11,0,False,False,False,False,False,True,False,False
4,5324,1981,4.0,58293.0,1,0,19-01-2014,94,173,43,...,11,0,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,2.0,61223.0,0,1,13-06-2013,46,709,43,...,11,0,False,False,False,True,False,False,False,False
2236,4001,1946,4.0,64014.0,2,1,10-06-2014,56,406,0,...,11,0,False,False,False,False,False,True,False,False
2237,7270,1981,2.0,56981.0,0,0,25-01-2014,91,908,48,...,11,0,False,False,True,False,False,False,False,False
2238,8235,1956,3.0,69245.0,0,1,24-01-2014,8,428,30,...,11,0,False,False,False,False,False,True,False,False


ya hemos limpiado las columnas categoricas, vamos a ver la columna con formato fecha

In [51]:
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'], format= "%d-%m-%Y") # paso la columna a formato datetime

# ahora el tema es tal como sigue, DBScan, que es el modelo de clusterizacion al que nos dirigimos, no acepta formato datetime, que vamos a hacer?
# vamos a pasar la columna datetima a horas y jugar con esa columna en formato numerico

In [53]:
# ojo que vienen curvas, 
# cojo una fecha de referencia, que sera la fecha minima de la columna Dt_customer y calculo la q de horas que ha transcurrido para cada instancia del data desde esa fecha de referencia
# asi transformaremos la columna a int

# para esto utilizaremos la funcion timedelta de pandas, que nos permite definir en distintas medidas temporales una cantidad definida por datetime, en nuestro caso lo haremos en horas

reference_time = data['Dt_Customer'].min()

data['Dt_Customer_hours'] = (data['Dt_Customer'] - reference_time) / pd.Timedelta(hours= 1)

In [54]:
data['Dt_Customer_hours'].value_counts() # comparo (siguiente celda) y parece que ha salido bien

Dt_Customer_hours
768.0      12
1056.0     11
4776.0     11
15624.0    11
9264.0     10
           ..
144.0       1
2664.0      1
7176.0      1
6192.0      1
12672.0     1
Name: count, Length: 663, dtype: int64

In [55]:
data['Dt_Customer'].value_counts() # ahora voy a quitarme esta columna de enmedio que me sobra

Dt_Customer
2012-08-31    12
2012-09-12    11
2013-02-14    11
2014-05-12    11
2013-08-20    10
              ..
2012-08-05     1
2012-11-18     1
2013-05-25     1
2013-04-14     1
2014-01-09     1
Name: count, Length: 663, dtype: int64

In [56]:
data.drop(['Dt_Customer'], axis= 1, inplace= True)

In [57]:
data.info() # pues ya lo tenemos, todas las filas en numericas, aexcepcion de los booleanos del one hot 

# ahora vamos a standarizar todo esto para que el DBScan pueda procesarlo bien, con los booleanos no deberia dar problemas, deberia interpretar 0 y 1, si da problema lo mapeo y fuera

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       2240 non-null   int64  
 1   Year_Birth               2240 non-null   int64  
 2   Education                2240 non-null   float64
 3   Income                   2240 non-null   float64
 4   Kidhome                  2240 non-null   int64  
 5   Teenhome                 2240 non-null   int64  
 6   Recency                  2240 non-null   int64  
 7   MntWines                 2240 non-null   int64  
 8   MntFruits                2240 non-null   int64  
 9   MntMeatProducts          2240 non-null   int64  
 10  MntFishProducts          2240 non-null   int64  
 11  MntSweetProducts         2240 non-null   int64  
 12  MntGoldProds             2240 non-null   int64  
 13  NumDealsPurchases        2240 non-null   int64  
 14  NumWebPurchases         

In [59]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(data)

In [61]:
df_scaled_data = pd.DataFrame(scaled_data, columns= data.columns) # mantenemos las columnas que teniamos antes que al escalar y pasar el array se han ido

In [62]:
df_scaled_data # so far so good

,ID,Year_Birth,Education,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,Response,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO,Dt_Customer_hours
0,-0.020999,-0.985345,-0.458383,0.235327,-0.825218,-0.929894,0.307039,0.983781,1.551577,1.679702,...,2.388846,-0.029894,-0.036621,-0.339909,-0.792406,1.914854,-0.591099,-0.188676,-0.029894,-1.531185
1,-1.053058,-1.235733,-0.458383,-0.235826,1.032559,0.906934,-0.383664,-0.870479,-0.636301,-0.713225,...,-0.418612,-0.029894,-0.036621,-0.339909,-0.792406,1.914854,-0.591099,-0.188676,-0.029894,1.190545
2,-0.447070,-0.317643,-0.458383,0.773633,-0.825218,-0.929894,-0.798086,0.362723,0.570804,-0.177032,...,-0.418612,-0.029894,-0.036621,-0.339909,-0.792406,-0.522233,1.691765,-0.188676,-0.029894,0.205773
3,0.181716,1.268149,-0.458383,-1.022732,1.032559,-0.929894,-0.798086,-0.870479,-0.560857,-0.651187,...,-0.418612,-0.029894,-0.036621,-0.339909,-0.792406,-0.522233,1.691765,-0.188676,-0.029894,1.061881
4,-0.082614,1.017761,1.533425,0.241519,1.032559,-0.929894,1.550305,-0.389085,0.419916,-0.216914,...,-0.418612,-0.029894,-0.036621,-0.339909,1.261980,-0.522233,-0.591099,-0.188676,-0.029894,0.953012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,1.625983,-0.150717,-0.458383,0.358568,-0.825218,0.906934,-0.107383,1.203678,0.419916,0.066692,...,-0.418612,-0.029894,-0.036621,-0.339909,1.261980,-0.522233,-0.591099,-0.188676,-0.029894,-0.135680
2236,-0.490200,-1.903435,1.533425,0.470064,2.890335,0.906934,0.237969,0.303291,-0.661449,-0.606873,...,-0.418612,-0.029894,-0.036621,-0.339909,-0.792406,-0.522233,1.691765,-0.188676,-0.029894,1.655713
2237,0.516905,1.017761,-0.458383,0.189106,-0.825218,-0.929894,1.446700,1.795020,0.545656,0.221789,...,-0.418612,-0.029894,-0.036621,2.941967,-0.792406,-0.522233,-0.591099,-0.188676,-0.029894,0.982703
2238,0.814199,-1.068807,0.537521,0.679035,-0.825218,0.906934,-1.419719,0.368666,0.092992,0.208495,...,-0.418612,-0.029894,-0.036621,-0.339909,-0.792406,-0.522233,1.691765,-0.188676,-0.029894,0.977755


In [63]:
df_scaled_data.describe()

# media 0 desviacion 1, todo parece bien por ahora, me inquieta que la desviacion de todas las columnas es la misma (1.000223e+00), es esto normal o se me escapa algo?

# reviso y no encuentro lagunas, debe de estar bien

# de momento va a quedar asi, pero si encuentro cosas raras este va a ser un punto de estudio

,ID,Year_Birth,Education,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,...,Response,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO,Dt_Customer_hours
count,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,...,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03,2.240000e+03
mean,2.220446e-17,-8.133970e-15,3.489272e-17,5.154607e-17,-1.586033e-17,-3.172066e-18,-3.172066e-18,-7.612958e-17,-2.379049e-17,4.123686e-17,...,-7.295751e-17,3.172066e-18,-3.172066e-18,-4.996004e-17,-6.344132e-17,5.392512e-17,7.137148e-17,-3.647876e-17,3.172066e-18,8.723181e-17
std,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,...,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00,1.000223e+00
min,-1.722818e+00,-6.326960e+00,-2.450190e+00,-2.018090e+00,-8.252176e-01,-9.298944e-01,-1.696001e+00,-9.031668e-01,-6.614492e-01,-7.398135e-01,...,-4.186121e-01,-2.989406e-02,-3.662078e-02,-3.399086e-01,-7.924058e-01,-5.222330e-01,-5.910986e-01,-1.886762e-01,-2.989406e-02,-1.709335e+00
25%,-8.514982e-01,-8.184192e-01,-4.583826e-01,-6.674801e-01,-8.252176e-01,-9.298944e-01,-8.671566e-01,-8.325919e-01,-6.363012e-01,-6.689119e-01,...,-4.186121e-01,-2.989406e-02,-3.662078e-02,-3.399086e-01,-7.924058e-01,-5.222330e-01,-5.910986e-01,-1.886762e-01,-2.989406e-02,-8.680727e-01
50%,-4.117757e-02,9.967091e-02,-4.583826e-01,-2.020403e-02,-8.252176e-01,-9.298944e-01,-3.777284e-03,-3.875991e-01,-4.602650e-01,-4.429132e-01,...,-4.186121e-01,-2.989406e-02,-3.662078e-02,-3.399086e-01,-7.924058e-01,-5.222330e-01,-5.910986e-01,-1.886762e-01,-2.989406e-02,-9.490706e-03
75%,8.735813e-01,6.839101e-01,5.375214e-01,6.408743e-01,1.032559e+00,9.069340e-01,8.596020e-01,5.952483e-01,1.684356e-01,2.882592e-01,...,-4.186121e-01,-2.989406e-02,-3.662078e-02,-3.399086e-01,1.261980e+00,-5.222330e-01,1.691765e+00,-1.886762e-01,-2.989406e-02,8.552770e-01
max,1.724876e+00,2.269702e+00,1.533425e+00,2.454513e+01,2.890335e+00,2.743762e+00,1.722981e+00,3.533390e+00,4.343008e+00,6.904261e+00,...,2.388846e+00,3.345146e+01,2.730690e+01,2.941967e+00,1.261980e+00,1.914854e+00,1.691765e+00,5.300086e+00,3.345146e+01,1.749736e+00


hemos acabado nuestra limpieza, vamos a guardar este data en un nuevo csv y recuperarlo desde el siguiente notebook, donde realizaremos una seleccion de features

haremos nuestro eda y estudiaremos correlaciones y colinealidades a ver que podemos dejar y quitar

a continuacion, con el data resultante, reduciremos aun mas la dimensionalidad utilizando PCA, pretendo simplificar y hacer mas eficiente todo el proyecto, ademas facilitara la  
visualizacion de los clusteres en las etapas finales del proyecto, pero no adelantemos acontecimientos

In [64]:
df_scaled_data.to_csv('./data/df_scaled_data.csv')